In [1]:
import numpy as np
import pandas as pd
#import Auxillary_Functions as af
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yaml
import math

environment_path = 'environment.yml'
#af.read_data(environment_path)

from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

from sklearn import preprocessing
from sklearn import datasets, linear_model

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit, \
        validation_curve, cross_validate, train_test_split, KFold, cross_val_predict
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelBinarizer, RobustScaler, PolynomialFeatures

# Load csvs from path and provide summary statistics
#file_path_1 = 'C:\\Users\\josephMNjerenga1\\Desktop\\Python_class\\Bike-Sharing-Dataset\\day.csv'
#file_path_2 = 'C:\\Users\\josephMNjerenga1\\Desktop\\Python_class\\Bike-Sharing-Dataset\\hour.csv'

original_day = pd.read_csv('C:\\Users\\josephMNjerenga1\\Desktop\\Python_class\\Bike-Sharing-Dataset\\day.csv', index_col=None)
original_hour= pd.read_csv('C:\\Users\\josephMNjerenga1\\Desktop\\Python_class\\Bike-Sharing-Dataset\\hour.csv', index_col=None)
    

In [2]:
raw_day_df=original_day.copy()
raw_day_df.describe(include = 'all')

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
unique,NaN,731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2011-09-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,366.000000,NaN,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,NaN,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,NaN,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,NaN,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,NaN,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,NaN,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000


In [3]:
raw_hour_df=original_hour.copy()
raw_hour_df.describe(include = 'all')

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
unique,NaN,731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2012-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,8690.0000,NaN,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,5017.0295,NaN,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.0000,NaN,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,NaN,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,NaN,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,NaN,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000


# Exploratory Data Analysis

In [4]:
raw_day_df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [5]:
raw_hour_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [6]:
raw_day_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
instant       731 non-null int64
dteday        731 non-null object
season        731 non-null int64
yr            731 non-null int64
mnth          731 non-null int64
holiday       731 non-null int64
weekday       731 non-null int64
workingday    731 non-null int64
weathersit    731 non-null int64
temp          731 non-null float64
atemp         731 non-null float64
hum           731 non-null float64
windspeed     731 non-null float64
casual        731 non-null int64
registered    731 non-null int64
cnt           731 non-null int64
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


In [7]:
raw_hour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
instant       17379 non-null int64
dteday        17379 non-null object
season        17379 non-null int64
yr            17379 non-null int64
mnth          17379 non-null int64
hr            17379 non-null int64
holiday       17379 non-null int64
weekday       17379 non-null int64
workingday    17379 non-null int64
weathersit    17379 non-null int64
temp          17379 non-null float64
atemp         17379 non-null float64
hum           17379 non-null float64
windspeed     17379 non-null float64
casual        17379 non-null int64
registered    17379 non-null int64
cnt           17379 non-null int64
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [8]:
raw_day_df.isnull().values.any()

False

In [9]:
raw_hour_df.isnull().values.any()

False

# Data Cleaning and Preparation

## Checking for correctness in dates

In [10]:
raw_hour_df['dteday'] = pd.to_datetime(raw_hour_df.dteday)

First check if column 'mnth' corresponds to the month value from 'dteday' column

In [11]:
all(raw_hour_df['dteday'].dt.month == raw_hour_df['mnth'])

True

Then check if column 'yr' corresponds to the year values from 'dteday' column represented as binary

In [12]:
binary_year = pd.Series(raw_hour_df['dteday'].dt.year == 2012)
all(binary_year == raw_hour_df['yr'].astype('bool'))

True

Finally, create a 'day' column that represents the day of the month

In [13]:
raw_hour_df[(raw_hour_df.season == 4) & (raw_hour_df.mnth == 12)].head(2)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
7904,7905,2011-12-01,4,0,12,0,0,4,1,1,0.28,0.2576,0.52,0.3284,1,19,20
7905,7906,2011-12-01,4,0,12,1,0,4,1,1,0.26,0.2424,0.60,0.2836,1,9,10


Defining previous steps as function to later use in data preparation pipeline:

## Data preparation 

### Data preparation functions

In [14]:
def fix_types(df):
    df[['season','mnth', 'hr', 'weekday','weathersit']] = df[['season','mnth', 'hr', 'weekday','weathersit']].astype('category')
    df[['yr', 'holiday', 'workingday']] = df[['yr', 'holiday', 'workingday']].astype('uint8')
    return df

In [15]:
def feature_skewness(df):
    numeric_dtypes = ['float64'] ##
    numeric_features = []
    for i in df.columns:
        if df[i].dtype in numeric_dtypes: 
            numeric_features.append(i)

    feature_skew = df[numeric_features].apply(
        lambda x: skew(x)).sort_values(ascending=False)
    skews = pd.DataFrame({'skew':feature_skew})
    return feature_skew, numeric_features

def fix_skewness(df):
    feature_skew, numeric_features = feature_skewness(df)
    high_skew = feature_skew[feature_skew > 0.5]
    skew_index = high_skew.index
    
    for i in skew_index:
        df[i] = boxcox1p(df[i], boxcox_normmax(df[i]+1))

    skew_features = df[numeric_features].apply(
        lambda x: skew(x)).sort_values(ascending=False)
    skews = pd.DataFrame({'skew':skew_features})
    return df

In [16]:
def numerical_features(df):
    columns = df.columns
    return df._get_numeric_data().columns

def categorical_features(df):
    numerical_columns = numerical_features(df)
    return(list(set(df.columns) - set(numerical_columns)))

def onehot_encode(df):
    numericals = df.get(numerical_features(df))
    new_df = numericals.copy()
    for categorical_column in categorical_features(df):
        new_df = pd.concat([new_df, 
                            pd.get_dummies(df[categorical_column], 
                                           prefix=categorical_column)], 
                           axis=1)
    return new_df

### Data Preparation Pipeline

In [17]:
#####Skewness????????

In [18]:
def standardize_data(df):
    std_scale = preprocessing.MinMaxScaler().fit(df[['atemp', 'hum', 'windspeed']])
    df[['atemp', 'hum', 'windspeed']] = std_scale.transform(df[['atemp', 'hum', 'windspeed']])
    
    return df

In [19]:
def standardize_hour(df):
    df[['hr']] = df[['hr']].astype('category')
    df = onehot_encode(df)
    
    return df

In [20]:
def drop_columns(df):
    df = df.drop(['instant'], axis = 1)
    return df

In [21]:
dataset_fixed = fix_types(fix_skewness(standardize_data(drop_columns(raw_hour_df))))
dataset_encoded=onehot_encode(dataset_fixed)

In [22]:
dataset_fixed.head(2)

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40


In [23]:
dataset_encoded.head(2)

,yr,holiday,workingday,temp,atemp,hum,windspeed,casual,registered,cnt,...,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12
0,0,0,0,0.24,0.2879,0.81,0.0,3,13,16,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0.22,0.2727,0.80,0.0,8,32,40,...,0,0,0,0,0,0,0,0,0,0


# Modelling

## 1. Baseline model

Split in test and train set

In [24]:
test = dataset_fixed[(dataset_fixed.yr == 1) & ((dataset_fixed.dteday.dt.month == 10)|(dataset_fixed.dteday.dt.month == 11)|(dataset_fixed.dteday.dt.month == 12))]
train = dataset_fixed[:(len(dataset_fixed)-len(test))]

test = onehot_encode(test.drop(['dteday'], axis = 1))
train = onehot_encode(train.drop(['dteday'], axis = 1))

In [25]:
test.head(2)

,yr,holiday,workingday,temp,atemp,hum,windspeed,casual,registered,cnt,...,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12
15211,1,0,1,0.46,0.4545,0.72,0.114076,6,39,45,...,0,0,0,0,0,0,0,1,0,0
15212,1,0,1,0.44,0.4394,0.77,0.098804,5,13,18,...,0,0,0,0,0,0,0,1,0,0


In [26]:
train.tail(2)

,yr,holiday,workingday,temp,atemp,hum,windspeed,casual,registered,cnt,...,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12
15209,1,0,0,0.48,0.4697,0.72,0.000000,15,76,91,...,0,0,0,0,0,0,1,0,0,0
15210,1,0,0,0.48,0.4697,0.72,0.098804,8,49,57,...,0,0,0,0,0,0,1,0,0,0


Defining the linear regression model for the baseline score

In [27]:
def score_model(train, seed = 123):
    X = train.loc[:, (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ]
    y = train.loc[:, 'cnt']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                        random_state=seed)        
                        
    # Create linear regression object
    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = [1 if i < 0 else math.ceil(i) for i in regr.predict(X_test)]
    return r2_score(y_test, y_pred)
   

In [28]:
from sklearn.model_selection import TimeSeriesSplit
def score_model(train, test, seed = 123):
    X_train = train.loc[:, (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ]
    y_train = train.loc[:, 'cnt']
    
    X_test = test.loc[:, (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ]
    y_test = test.loc[:, 'cnt']             
                        
    cv  = TimeSeriesSplit(n_splits= 5)

    pipeline = Pipeline([
        #('polynomials', PolynomialFeatures(degree=3, include_bias=False)), 
        ('linear_regression', LinearRegression(n_jobs=-1))])
    pipeline.fit(X_train, y_train)
    
    training_score = pipeline.score(X_test, y_test)
    print ('R2 best features')
    print('R2 from entire-dataset estimator: {:.2f}'.format(training_score))

    # Obtain scores and estimators from different splits and use the best one.
    scores = cross_validate(pipeline, 
                            X_train, y_train,
                            scoring=['r2'], 
                            cv=cv,
                            return_estimator=True)
    split_scores = [scores['estimator'][i].score(X_test, y_test) 
                    for i in range(len(scores))]
    index_best = split_scores.index(max(split_scores))
    print('Best estimator R2 score: {:.2f}'.format(split_scores[index_best]))

In [29]:
score_model(train, test)

R2 best features
R2 from entire-dataset estimator: 0.64
Best estimator R2 score: 0.63


## 2. Feature Engineering (FE)

### 2. FE1: Dropping atemp and weekday

The temperature variable is captured by the atemp variable, therefore we drop one of the two; in this case temperature.
The workingday variable is also able to capture similar information as the weekday variable, and so we drop weekday.

In [30]:
# Drop variables
dataset_FE1 = dataset_fixed.drop(['temp','weekday'], axis = 1)

In [31]:
dataset_FE1.head(2)

,dteday,season,yr,mnth,hr,holiday,workingday,weathersit,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,0,1,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,0,1,0.2727,0.80,0.0,8,32,40


In [32]:
test = dataset_FE1[(dataset_FE1.yr == 1) & ((dataset_FE1.dteday.dt.month == 10)|(dataset_FE1.dteday.dt.month == 11)|(dataset_FE1.dteday.dt.month == 12))]
train = dataset_FE1[:(len(dataset_FE1)-len(test))]

test = onehot_encode(test.drop(['dteday'], axis = 1))
train = onehot_encode(train.drop(['dteday'], axis = 1))

In [33]:
score_model(train, test)

R2 best features
R2 from entire-dataset estimator: 0.63
Best estimator R2 score: 0.62


### 2. FE2: Grouping hours and temperature

In [34]:
# Group hours based on pattern for casual users
dataset_FE2 = dataset_FE1.copy()
dataset_FE2.head(2)

,dteday,season,yr,mnth,hr,holiday,workingday,weathersit,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,0,1,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,0,1,0.2727,0.80,0.0,8,32,40


In [35]:
dataset_FE2['hour'] = pd.cut(dataset_FE2['hr'],[-1,6,12,17,23], labels=["Early_Morning", "Morning", "Afternon","Evening"])

In [36]:
dataset_FE2.head(2)

,dteday,season,yr,mnth,hr,holiday,workingday,weathersit,atemp,hum,windspeed,casual,registered,cnt,hour
0,2011-01-01,1,0,1,0,0,0,1,0.2879,0.81,0.0,3,13,16,Early_Morning
1,2011-01-01,1,0,1,1,0,0,1,0.2727,0.80,0.0,8,32,40,Early_Morning


In [37]:
dataset_FE2.hour.unique()

[Early_Morning, Morning, Afternon, Evening]
Categories (4, object): [Early_Morning < Morning < Afternon < Evening]

In [38]:
dataset_FE2["hour"] = dataset_FE2["hour"].astype('category').cat.codes
dataset_FE2["hour"] = dataset_FE2["hour"].astype('category')

In [39]:
dataset_FE2.hour.unique()

[0, 1, 2, 3]
Categories (4, int64): [0, 1, 2, 3]

In [40]:
dataset_FE2['atemp_level']=pd.cut(dataset_FE2['atemp'],3, labels=["cold", "regular", "hot"])
dataset_FE2["atemp_level"] = dataset_FE2["atemp_level"].astype('category').cat.codes
dataset_FE2["atemp_level"] = dataset_FE2["atemp_level"].astype('category')

In [41]:
dataset_FE2.head(2)

,dteday,season,yr,mnth,hr,holiday,workingday,weathersit,atemp,hum,windspeed,casual,registered,cnt,hour,atemp_level
0,2011-01-01,1,0,1,0,0,0,1,0.2879,0.81,0.0,3,13,16,0,0
1,2011-01-01,1,0,1,1,0,0,1,0.2727,0.80,0.0,8,32,40,0,0


In [42]:
test = dataset_FE2[(dataset_FE2.yr == 1) & ((dataset_FE2.dteday.dt.month == 10)|(dataset_FE2.dteday.dt.month == 11)|(dataset_FE2.dteday.dt.month == 12))]
train = dataset_FE2[:(len(dataset_FE2)-len(test))]

test = onehot_encode(test.drop(['dteday'], axis = 1))
train = onehot_encode(train.drop(['dteday'], axis = 1))

In [43]:
score_model(train, test)

R2 best features
R2 from entire-dataset estimator: 0.63
Best estimator R2 score: 0.62


### 2. FE2: Drop month but keep seasons

In [44]:
dataset_FE3 = dataset_fixed.drop(['atemp', 'weekday', 'mnth'], axis = 1)

In [45]:
test = dataset_FE3[(dataset_FE3.yr == 1) & ((dataset_FE3.dteday.dt.month == 10)|(dataset_FE3.dteday.dt.month == 11)|(dataset_FE3.dteday.dt.month == 12))]
train = dataset_FE3[:(len(dataset_FE3)-len(test))]

test = onehot_encode(test.drop(['dteday'], axis = 1))
train = onehot_encode(train.drop(['dteday'], axis = 1))

In [46]:
score_model(train, test)

R2 best features
R2 from entire-dataset estimator: 0.64
Best estimator R2 score: 0.62


## 3. Gradient Boosting - RSME

In [47]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

X_train = train.loc[:, (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ]
y_train = train.loc[:, 'cnt']
    
X_test = test.loc[:, (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ]
y_test = test.loc[:, 'cnt']   

gbm = GradientBoostingRegressor(n_estimators=4000,alpha=0.01)
gbm.fit(X = X_train,y = y_train)
preds = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in gbm.predict(X= X_test)], index = X_test.index.values)
print ("RMSE Value: {:.3f} ".format(mean_squared_error(y_test, preds)))
print ("R2 score: {:.3f} ".format(r2_score(preds, y_test)))

RMSE Value: 4729.387 
R2 score: 0.853 


## 4. Gradient Boosting - RMSLE

In [49]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=4000,alpha=0.01); ### Test 0.41

def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

X_train = train.loc[:, (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered')]
y_train = train.loc[:, 'cnt']
    
X_test = test.loc[:, (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered')]
y_test = test.loc[:, 'cnt'] 

y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

gbm.fit(X_train, y_train_log)
preds = gbm.predict(X = X_test)
print ("RMSLE Value For Gradient Boost: {:.3f} ".format(rmsle(np.exp(y_test_log),np.exp(preds),False)))
print ("R2 score: {:.3f} ".format(r2_score(preds, y_test_log)))

RMSLE Value For Gradient Boost: 0.400 
R2 score: 0.905 


In [50]:
r2_score(pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in np.exp(preds)]),y_test)

0.8664192592187296

## 5. Regularization Model - Ridge

In [51]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

X_train = train.loc[:, (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered')]
y_train = train.loc[:, 'cnt']
    
X_test = test.loc[:, (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered')]
y_test = test.loc[:, 'cnt'] 

ridge_m_ = Ridge()
ridge_params_ = { 'max_iter':[3000],'alpha':[0.1, 1, 2, 3, 4, 10, 30,100,200,300,400,800,900,1000]}
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
grid_ridge_m = GridSearchCV( ridge_m_,
                          ridge_params_,
                          scoring = rmsle_scorer,
                          cv=5)

y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

grid_ridge_m.fit( X_train, y_train_log )
preds = grid_ridge_m.predict(X= X_test)
print (grid_ridge_m.best_params_)
print ("RMSLE Value For Ridge Regression: {:.3f} ".format(rmsle(np.exp(y_test_log),np.exp(preds),False)))
print ("R2 score: {:.3f} ".format(r2_score(preds, y_test_log)))

#fig,ax= plt.subplots()
#fig.set_size_inches(12,5)
#df = pd.DataFrame(grid_ridge_m.grid_scores_)
#df["alpha"] = df["parameters"].apply(lambda x:x["alpha"])
#df["rmsle"] = df["mean_validation_score"].apply(lambda x:-x)
#sn.pointplot(data=df,x="alpha",y="rmsle",ax=ax)

{'alpha': 1, 'max_iter': 3000}
RMSLE Value For Ridge Regression: 0.597 
R2 score: 0.744 


In [52]:
r2_score(pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in np.exp(preds)]),y_test)

0.5319325374517861

## 6. Regularization - LASSO

In [53]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn import metrics
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

X_train = train.loc[:, (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered')]
y_train = train.loc[:, 'cnt']
    
X_test = test.loc[:, (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered')]
y_test = test.loc[:, 'cnt'] 

lasso_m_ = Lasso()

alpha  = 1/np.array([0.1, 1, 2, 3, 4, 10, 30,100,200,300,400,800,900,1000])
lasso_params_ = { 'max_iter':[3000],'alpha':alpha}

grid_lasso_m = GridSearchCV( lasso_m_,lasso_params_,scoring = rmsle_scorer,cv=5)

y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

grid_lasso_m.fit( X_train, y_train_log )
preds = grid_lasso_m.predict(X= X_test)
print (grid_lasso_m.best_params_)
print ("RMSLE Value For Lasso Regression: {:.3f} ".format(rmsle(np.exp(y_test_log),np.exp(preds),False)))
print ("R2 score: {:.3f} ".format(r2_score(preds, y_test_log)))

#fig,ax= plt.subplots()
#fig.set_size_inches(12,5)
#df = pd.DataFrame(grid_lasso_m.grid_scores_)
#df["alpha"] = df["parameters"].apply(lambda x:x["alpha"])
#df["rmsle"] = df["mean_validation_score"].apply(lambda x:-x)
#sn.pointplot(data=df,x="alpha",y="rmsle",ax=ax)

{'alpha': 0.001, 'max_iter': 3000}
RMSLE Value For Lasso Regression: 0.598 
R2 score: 0.735 


## 7. Random Forest

In [54]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

X_train = train.loc[:, (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ]
y_train = train.loc[:, 'cnt']
    
X_test = test.loc[:, (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ]
y_test = test.loc[:, 'cnt']

rfModel = RandomForestRegressor(n_estimators=100)

y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

rfModel.fit(X_train,y_train_log)
preds = rfModel.predict(X = X_test)
print ("RMSLE Value For Random Forest: {:.3f} ".format(rmsle(np.exp(y_test_log),np.exp(preds),False)))
print ("R2 score: {:.3f} ".format(r2_score(preds, y_test_log)))

RMSLE Value For Random Forest: 0.477 
R2 score: 0.867 


## 8. Multiple models - data prep

In [55]:
def fix_types(df):
    df[['season','weathersit', 'mnth', 'weekday']] = df[['season','weathersit', 'mnth', 'weekday']].astype('category')
    df[['yr', 'holiday', 'workingday']] = df[['yr', 'holiday', 'workingday']].astype('uint8')
    df['dteday'] = pd.to_datetime(df.dteday)
    return df

In [56]:
def drop_columns(df):
    df = df.drop(['instant'], axis = 1)
    return df

In [57]:
def standardize_hour(df):
    df[['hr']] = df[['hr']].astype('category')
    df = onehot_encode(df)
    
    return df

In [58]:
dataset_fixed = fix_types(fix_skewness(drop_columns(raw_hour_df)))

In [59]:
dataset_fixed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 16 columns):
dteday        17379 non-null datetime64[ns]
season        17379 non-null category
yr            17379 non-null uint8
mnth          17379 non-null category
hr            17379 non-null int64
holiday       17379 non-null uint8
weekday       17379 non-null category
workingday    17379 non-null uint8
weathersit    17379 non-null category
temp          17379 non-null float64
atemp         17379 non-null float64
hum           17379 non-null float64
windspeed     17379 non-null float64
casual        17379 non-null int64
registered    17379 non-null int64
cnt           17379 non-null int64
dtypes: category(4), datetime64[ns](1), float64(4), int64(4), uint8(3)
memory usage: 1.3 MB


In [60]:
test = dataset_fixed[(dataset_fixed.yr == 1) & ((dataset_fixed.dteday.dt.month == 10)|(dataset_fixed.dteday.dt.month == 11)|(dataset_fixed.dteday.dt.month == 12))]
train = dataset_fixed[:(len(dataset_fixed)-len(test))]

test = onehot_encode(test.drop(['dteday', 'mnth', 'atemp'], axis = 1))
train = onehot_encode(train.drop(['dteday', 'mnth', 'atemp'], axis = 1))

## 8a. Original hours segmented

In [61]:
def score_multiple_original(train, test):
    
    X005_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr < 6), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y005_train = train.loc[(train.workingday == 0) & (train.hr < 6), 'cnt']
    X005_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr < 6), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X0512_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 5) & (train.hr < 13), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y0512_train = train.loc[(train.workingday == 0) & (train.hr > 5) & (train.hr < 13), 'cnt']
    X0512_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 5) & (test.hr < 13), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01216_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 12) & (train.hr < 17), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01216_train = train.loc[(train.workingday == 0) & (train.hr > 12) & (train.hr < 17), 'cnt']
    X01216_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 12) & (test.hr < 17), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01623_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 16), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01623_train = train.loc[(train.workingday == 0) & (train.hr > 16), 'cnt']
    X01623_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 16), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X104_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr < 5), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y104_train = train.loc[(train.workingday == 1) & (train.hr < 5), 'cnt']
    X104_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr < 5), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X148_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 4) & (train.hr < 9), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y148_train = train.loc[(train.workingday == 1) & (train.hr > 4) & (train.hr < 9), 'cnt']
    X148_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 4) & (test.hr < 9), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X1810_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 11), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y1810_train = train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 11), 'cnt']
    X1810_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 8) & (test.hr < 11), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11015_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 10) & (train.hr < 16), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11015_train = train.loc[(train.workingday == 1) & (train.hr > 10) & (train.hr < 16), 'cnt']
    X11015_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 10) & (test.hr < 16), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11518_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 15) & (train.hr < 19), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11518_train = train.loc[(train.workingday == 1) & (train.hr > 15) & (train.hr < 19), 'cnt']
    X11518_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 15) & (test.hr < 19), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11823_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 18), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11823_train = train.loc[(train.workingday == 1) & (train.hr > 18), 'cnt']
    X11823_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 18), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    y_test = test.loc[:, 'cnt']    
                        
    # Create linear regression objects
    regr005 = linear_model.LinearRegression()
    regr005.fit(X005_train, y005_train)
    y005_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr005.predict(X005_test)], index = X005_test.index.values)

    regr0512 = linear_model.LinearRegression()
    regr0512.fit(X0512_train, y0512_train)
    y0512_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr0512.predict(X0512_test)], index = X0512_test.index.values)

    regr01216 = linear_model.LinearRegression()
    regr01216.fit(X01216_train, y01216_train)
    y01216_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01216.predict(X01216_test)], index = X01216_test.index.values)

    regr01623 = linear_model.LinearRegression()
    regr01623.fit(X01623_train, y01623_train)
    y01623_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01623.predict(X01623_test)], index = X01623_test.index.values)

    regr104 = linear_model.LinearRegression()
    regr104.fit(X104_train, y104_train)
    y104_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr104.predict(X104_test)], index = X104_test.index.values)

    regr148 = linear_model.LinearRegression()
    regr148.fit(X148_train, y148_train)
    y148_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr148.predict(X148_test)], index = X148_test.index.values)

    regr1810 = linear_model.LinearRegression()
    regr1810.fit(X1810_train, y1810_train)
    y1810_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr1810.predict(X1810_test)], index = X1810_test.index.values)

    regr11015 = linear_model.LinearRegression()
    regr11015.fit(X11015_train, y11015_train)
    y11015_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11015.predict(X11015_test)], index = X11015_test.index.values)

    regr11518 = linear_model.LinearRegression()
    regr11518.fit(X11518_train, y11518_train)
    y11518_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11518.predict(X11518_test)], index = X11518_test.index.values)

    regr11823 = linear_model.LinearRegression()
    regr11823.fit(X11823_train, y11823_train)
    y11823_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11823.predict(X11823_test)], index = X11823_test.index.values)
    
    y_pred = pd.DataFrame(columns = list(y005_pred))
    y_pred = pd.concat([y005_pred, y0512_pred, y01216_pred, y01623_pred, y104_pred, y148_pred, y1810_pred, y11015_pred, y11518_pred, y11823_pred]).sort_index()
    
#     for i in y_test.index.values:
#         print((y_test[i], y_pred[i]))
    
    return r2_score(y_test, y_pred)
    



## 8b. Hours segments updated - SL and SN

In [62]:
def score_multiple2(train, test):
    
    X005_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr < 6), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y005_train = train.loc[(train.workingday == 0) & (train.hr < 6), 'cnt']
    X005_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr < 6), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X0512_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 5) & (train.hr < 13), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y0512_train = train.loc[(train.workingday == 0) & (train.hr > 5) & (train.hr < 13), 'cnt']
    X0512_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 5) & (test.hr < 13), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01216_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 12) & (train.hr < 17), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01216_train = train.loc[(train.workingday == 0) & (train.hr > 12) & (train.hr < 17), 'cnt']
    X01216_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 12) & (test.hr < 17), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01623_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 16), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01623_train = train.loc[(train.workingday == 0) & (train.hr > 16), 'cnt']
    X01623_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 16), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X104_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr < 5), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y104_train = train.loc[(train.workingday == 1) & (train.hr < 5), 'cnt']
    X104_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr < 5), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X148_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 4) & (train.hr < 9), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y148_train = train.loc[(train.workingday == 1) & (train.hr > 4) & (train.hr < 9), 'cnt']
    X148_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 4) & (test.hr < 9), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X1810_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 11), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y1810_train = train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 11), 'cnt']
    X1810_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 8) & (test.hr < 11), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11012_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 10) & (train.hr < 13), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11012_train = train.loc[(train.workingday == 1) & (train.hr > 10) & (train.hr < 13), 'cnt']
    X11012_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 10) & (test.hr < 13), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
   
    X11214_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 12) & (train.hr < 15), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11214_train = train.loc[(train.workingday == 1) & (train.hr > 12) & (train.hr < 15), 'cnt']
    X11214_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 12) & (test.hr < 15), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])

    X11415_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 14) & (train.hr < 16), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11415_train = train.loc[(train.workingday == 1) & (train.hr > 14) & (train.hr < 16), 'cnt']
    X11415_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 14) & (test.hr < 16), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])

    X11517_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 15) & (train.hr < 18), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11517_train = train.loc[(train.workingday == 1) & (train.hr > 15) & (train.hr < 18), 'cnt']
    X11517_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 15) & (test.hr < 18), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])

    X11718_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 17) & (train.hr < 19), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11718_train = train.loc[(train.workingday == 1) & (train.hr > 17) & (train.hr < 19), 'cnt']
    X11718_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 17) & (test.hr < 19), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11823_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 18), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11823_train = train.loc[(train.workingday == 1) & (train.hr > 18), 'cnt']
    X11823_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 18), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    y_test = test.loc[:, 'cnt']    
                        
    # Create linear regression objects
    regr005 = linear_model.LinearRegression()
    regr005.fit(X005_train, y005_train)
    y005_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr005.predict(X005_test)], index = X005_test.index.values)

    regr0512 = linear_model.LinearRegression()
    regr0512.fit(X0512_train, y0512_train)
    y0512_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr0512.predict(X0512_test)], index = X0512_test.index.values)

    regr01216 = linear_model.LinearRegression()
    regr01216.fit(X01216_train, y01216_train)
    y01216_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01216.predict(X01216_test)], index = X01216_test.index.values)

    regr01623 = linear_model.LinearRegression()
    regr01623.fit(X01623_train, y01623_train)
    y01623_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01623.predict(X01623_test)], index = X01623_test.index.values)

    regr104 = linear_model.LinearRegression()
    regr104.fit(X104_train, y104_train)
    y104_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr104.predict(X104_test)], index = X104_test.index.values)

    regr148 = linear_model.LinearRegression()
    regr148.fit(X148_train, y148_train)
    y148_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr148.predict(X148_test)], index = X148_test.index.values)

    regr1810 = linear_model.LinearRegression()
    regr1810.fit(X1810_train, y1810_train)
    y1810_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr1810.predict(X1810_test)], index = X1810_test.index.values)

    regr11012 = linear_model.LinearRegression()
    regr11012.fit(X11012_train, y11012_train)
    y11012_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11012.predict(X11012_test)], index = X11012_test.index.values)
    
    regr11214 = linear_model.LinearRegression()
    regr11214.fit(X11214_train, y11214_train)
    y11214_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11214.predict(X11214_test)], index = X11214_test.index.values)
    
    regr11415 = linear_model.LinearRegression()
    regr11415.fit(X11415_train, y11415_train)
    y11415_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11415.predict(X11415_test)], index = X11415_test.index.values)

    regr11517 = linear_model.LinearRegression()
    regr11517.fit(X11517_train, y11517_train)
    y11517_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11517.predict(X11517_test)], index = X11517_test.index.values)
    
    regr11718 = linear_model.LinearRegression()
    regr11718.fit(X11718_train, y11718_train)
    y11718_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11718.predict(X11718_test)], index = X11718_test.index.values)

    regr11823 = linear_model.LinearRegression()
    regr11823.fit(X11823_train, y11823_train)
    y11823_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11823.predict(X11823_test)], index = X11823_test.index.values)
    
    y_pred = pd.DataFrame(columns = list(y005_pred))
    y_pred = pd.concat([y005_pred, y0512_pred, y01216_pred, y01623_pred, y104_pred, y148_pred, y1810_pred, y11012_pred, y11214_pred, y11415_pred, y11517_pred, y11718_pred, y11823_pred]).sort_index()
    
#     for i in y_test.index.values:
#         print((y_test[i], y_pred[i]))
    
    return r2_score(y_test, y_pred)
    



## 8c. Hours segmented for casual bikers

In [63]:
def score_multiple_casual(train, test):
    
    X005_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr < 6), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y005_train = train.loc[(train.workingday == 0) & (train.hr < 6), 'casual']
    X005_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr < 6), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X058_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 5) & (train.hr < 9), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y058_train = train.loc[(train.workingday == 0) & (train.hr > 5) & (train.hr < 9), 'casual']
    X058_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 5) & (test.hr < 9), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X0814_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 8) & (train.hr < 15), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y0814_train = train.loc[(train.workingday == 0) & (train.hr > 8) & (train.hr < 15), 'casual']
    X0814_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 8) & (test.hr < 15), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01420_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 14) & (train.hr < 21), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01420_train = train.loc[(train.workingday == 0) & (train.hr > 14) & (train.hr < 21), 'casual']
    X01420_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 14) & (test.hr < 21), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
                                          
    X02023_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 20), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y02023_train = train.loc[(train.workingday == 0) & (train.hr > 20), 'casual']
    X02023_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 20), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X105_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr < 6), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y105_train = train.loc[(train.workingday == 1) & (train.hr < 6), 'casual']
    X105_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr < 6), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X158_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 5) & (train.hr < 9), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y158_train = train.loc[(train.workingday == 1) & (train.hr > 5) & (train.hr < 9), 'casual']
    X158_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 5) & (test.hr < 9), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X189_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 10), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y189_train = train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 10), 'casual']
    X189_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 8) & (test.hr < 10), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X1916_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 9) & (train.hr < 17), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y1916_train = train.loc[(train.workingday == 1) & (train.hr > 9) & (train.hr < 17), 'casual']
    X1916_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 9) & (test.hr < 17), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11617_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 16) & (train.hr < 18), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11617_train = train.loc[(train.workingday == 1) & (train.hr > 16) & (train.hr < 18), 'casual']
    X11617_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 16) & (test.hr < 18), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
   
    X11723_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 17), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11723_train = train.loc[(train.workingday == 1) & (train.hr > 17), 'casual']
    X11723_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 17), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    y_test = test.loc[:, 'casual']    
                        
    # Create linear regression objects
    regr005 = linear_model.LinearRegression()
    regr005.fit(X005_train, y005_train)
    y005_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr005.predict(X005_test)], index = X005_test.index.values)

    regr058 = linear_model.LinearRegression()
    regr058.fit(X058_train, y058_train)
    y058_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr058.predict(X058_test)], index = X058_test.index.values)

    regr0814 = linear_model.LinearRegression()
    regr0814.fit(X0814_train, y0814_train)
    y0814_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr0814.predict(X0814_test)], index = X0814_test.index.values)

    regr01420 = linear_model.LinearRegression()
    regr01420.fit(X01420_train, y01420_train)
    y01420_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01420.predict(X01420_test)], index = X01420_test.index.values)

    regr02023 = linear_model.LinearRegression()
    regr02023.fit(X02023_train, y02023_train)
    y02023_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr02023.predict(X02023_test)], index = X02023_test.index.values)
                                           
    regr105 = linear_model.LinearRegression()
    regr105.fit(X105_train, y105_train)
    y105_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr105.predict(X105_test)], index = X105_test.index.values)

    regr158 = linear_model.LinearRegression()
    regr158.fit(X158_train, y158_train)
    y158_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr158.predict(X158_test)], index = X158_test.index.values)

    regr189 = linear_model.LinearRegression()
    regr189.fit(X189_train, y189_train)
    y189_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr189.predict(X189_test)], index = X189_test.index.values)
                                           
    regr1916 = linear_model.LinearRegression()
    regr1916.fit(X1916_train, y1916_train)
    y1916_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr1916.predict(X1916_test)], index = X1916_test.index.values)

    regr11617 = linear_model.LinearRegression()
    regr11617.fit(X11617_train, y11617_train)
    y11617_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11617.predict(X11617_test)], index = X11617_test.index.values)
                                           
    regr11723 = linear_model.LinearRegression()
    regr11723.fit(X11723_train, y11723_train)
    y11723_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11723.predict(X11723_test)], index = X11723_test.index.values)


    y_pred = pd.DataFrame(columns = list(y005_pred))
    y_pred = pd.concat([y005_pred, y058_pred, y0814_pred, y01420_pred, y02023_pred, y105_pred, y158_pred, y189_pred, y1916_pred, y11617_pred, y11723_pred]).sort_index()
    
#     for i in y_test.index.values:
#         print((y_test[i], y_pred[i]))
    
    return r2_score(y_test, y_pred)
    



## 8d. Hours segmented for registered bikers

In [64]:
def score_multiple_registered(train, test):
    
    X005_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr < 6), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y005_train = train.loc[(train.workingday == 0) & (train.hr < 6), 'registered']
    X005_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr < 6), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X058_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 5) & (train.hr < 9), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y058_train = train.loc[(train.workingday == 0) & (train.hr > 5) & (train.hr < 9), 'registered']
    X058_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 5) & (test.hr < 9), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X0812_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 8) & (train.hr < 13), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y0812_train = train.loc[(train.workingday == 0) & (train.hr > 8) & (train.hr < 13), 'registered']
    X0812_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 8) & (test.hr < 13), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01216_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 12) & (train.hr < 17), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01216_train = train.loc[(train.workingday == 0) & (train.hr > 12) & (train.hr < 17), 'registered']
    X01216_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 12) & (test.hr < 17), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01620_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 16) & (train.hr < 21), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01620_train = train.loc[(train.workingday == 0) & (train.hr > 16) & (train.hr < 21), 'registered']
    X01620_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 16) & (test.hr < 21), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
                                          
    X02023_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 20), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y02023_train = train.loc[(train.workingday == 0) & (train.hr > 20), 'registered']
    X02023_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 20), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X105_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr < 6), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y105_train = train.loc[(train.workingday == 1) & (train.hr < 6), 'registered']
    X105_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr < 6), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X158_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 5) & (train.hr < 9), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y158_train = train.loc[(train.workingday == 1) & (train.hr > 5) & (train.hr < 9), 'registered']
    X158_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 5) & (test.hr < 9), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X1810_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 11), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y1810_train = train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 11), 'registered']
    X1810_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 8) & (test.hr < 11), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11012_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 10) & (train.hr < 13), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11012_train = train.loc[(train.workingday == 1) & (train.hr >10) & (train.hr < 13), 'registered']
    X11012_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 10) & (test.hr < 13), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11215_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 12) & (train.hr < 16), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11215_train = train.loc[(train.workingday == 1) & (train.hr >12) & (train.hr < 16), 'registered']
    X11215_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 12) & (test.hr < 16), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11517_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 15) & (train.hr < 18), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11517_train = train.loc[(train.workingday == 1) & (train.hr > 15) & (train.hr < 18), 'registered']
    X11517_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 15) & (test.hr < 18), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
   
    X11723_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 17), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11723_train = train.loc[(train.workingday == 1) & (train.hr > 17), 'registered']
    X11723_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 17), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    y_test = test.loc[:, 'registered']    
                        
    # Create linear regression objects
    regr005 = linear_model.LinearRegression()
    regr005.fit(X005_train, y005_train)
    y005_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr005.predict(X005_test)], index = X005_test.index.values)

    regr058 = linear_model.LinearRegression()
    regr058.fit(X058_train, y058_train)
    y058_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr058.predict(X058_test)], index = X058_test.index.values)

    regr0812 = linear_model.LinearRegression()
    regr0812.fit(X0812_train, y0812_train)
    y0812_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr0812.predict(X0812_test)], index = X0812_test.index.values)

    regr01216 = linear_model.LinearRegression()
    regr01216.fit(X01216_train, y01216_train)
    y01216_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01216.predict(X01216_test)], index = X01216_test.index.values)
    
    regr01620 = linear_model.LinearRegression()
    regr01620.fit(X01620_train, y01620_train)
    y01620_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01620.predict(X01620_test)], index = X01620_test.index.values)

    regr02023 = linear_model.LinearRegression()
    regr02023.fit(X02023_train, y02023_train)
    y02023_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr02023.predict(X02023_test)], index = X02023_test.index.values)
                                           
    regr105 = linear_model.LinearRegression()
    regr105.fit(X105_train, y105_train)
    y105_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr105.predict(X105_test)], index = X105_test.index.values)

    regr158 = linear_model.LinearRegression()
    regr158.fit(X158_train, y158_train)
    y158_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr158.predict(X158_test)], index = X158_test.index.values)

    regr1810 = linear_model.LinearRegression()
    regr1810.fit(X1810_train, y1810_train)
    y1810_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr1810.predict(X1810_test)], index = X1810_test.index.values)
                                           
    regr11012 = linear_model.LinearRegression()
    regr11012.fit(X11012_train, y11012_train)
    y11012_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11012.predict(X11012_test)], index = X11012_test.index.values)

    regr11215 = linear_model.LinearRegression()
    regr11215.fit(X11215_train, y11215_train)
    y11215_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11215.predict(X11215_test)], index = X11215_test.index.values)
    
    regr11517 = linear_model.LinearRegression()
    regr11517.fit(X11517_train, y11517_train)
    y11517_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11517.predict(X11517_test)], index = X11517_test.index.values)
                                           
    regr11723 = linear_model.LinearRegression()
    regr11723.fit(X11723_train, y11723_train)
    y11723_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11723.predict(X11723_test)], index = X11723_test.index.values)


    y_pred = pd.DataFrame(columns = list(y005_pred))
    y_pred = pd.concat([y005_pred, y058_pred, y0812_pred, y01216_pred, y01620_pred, y02023_pred, y105_pred, y158_pred, y1810_pred, y11012_pred, y11215_pred, y11517_pred, y11723_pred]).sort_index()
    
#     for i in y_test.index.values:
#         print((y_test[i], y_pred[i]))
    
    return r2_score(y_test, y_pred)
    



In [65]:
# 8a
score_multiple_original(train, test)

0.8477378338129921

In [66]:
# 8b
score_multiple2(train, test)

0.849009479440315

In [67]:
# 8c
score_multiple_casual(train, test)

0.7138795186345392

In [68]:
# 8d
score_multiple_registered(train, test)

0.8590341129503711

## 8e. Tomas' hours segmentation

In [69]:
def score_multiple4(train, test, model):
    
    X007_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr < 8), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y007_train = train.loc[(train.workingday == 0) & (train.hr < 8), 'cnt']
    X007_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr < 8), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X0710_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 7) & (train.hr < 11), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y0710_train = train.loc[(train.workingday == 0) & (train.hr > 7) & (train.hr < 11), 'cnt']
    X0710_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 7) & (test.hr < 11), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01017_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 10) & (train.hr < 18), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01017_train = train.loc[(train.workingday == 0) & (train.hr > 10) & (train.hr < 18), 'cnt']
    X01017_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 10) & (test.hr < 18), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X01723_train = standardize_hour(train.loc[(train.workingday == 0) & (train.hr > 17), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y01723_train = train.loc[(train.workingday == 0) & (train.hr > 17), 'cnt']
    X01723_test = standardize_hour(test.loc[(test.workingday == 0) & (test.hr > 17), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X106_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr < 7), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y106_train = train.loc[(train.workingday == 1) & (train.hr < 7), 'cnt']
    X106_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr < 7), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X168_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 6) & (train.hr < 9), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y168_train = train.loc[(train.workingday == 1) & (train.hr > 6) & (train.hr < 9), 'cnt']
    X168_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 6) & (test.hr < 9), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X1811_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 12), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y1811_train = train.loc[(train.workingday == 1) & (train.hr > 8) & (train.hr < 12), 'cnt']
    X1811_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 8) & (test.hr < 12), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11116_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 11) & (train.hr < 17), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11116_train = train.loc[(train.workingday == 1) & (train.hr > 11) & (train.hr < 17), 'cnt']
    X11116_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 11) & (test.hr < 17), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11618_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 16) & (train.hr < 19), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11618_train = train.loc[(train.workingday == 1) & (train.hr > 16) & (train.hr < 19), 'cnt']
    X11618_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 16) & (test.hr < 19), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    X11823_train = standardize_hour(train.loc[(train.workingday == 1) & (train.hr > 18), (train.columns != 'cnt') & (train.columns != 'casual') & (train.columns != 'registered') ])
    y11823_train = train.loc[(train.workingday == 1) & (train.hr > 18), 'cnt']
    X11823_test = standardize_hour(test.loc[(test.workingday == 1) & (test.hr > 18), (test.columns != 'cnt') & (test.columns != 'casual') & (test.columns != 'registered') ])
    
    y_test = test.loc[:, 'cnt']    
                        
    # Create linear regression objects
    regr007 = model
    regr007.fit(X007_train, y007_train)
    y007_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr007.predict(X007_test)], index = X007_test.index.values)

    regr0710 = model
    regr0710.fit(X0710_train, y0710_train)
    y0710_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr0710.predict(X0710_test)], index = X0710_test.index.values)

    regr01017 = model
    regr01017.fit(X01017_train, y01017_train)
    y01017_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01017.predict(X01017_test)], index = X01017_test.index.values)

    regr01723 = model
    regr01723.fit(X01723_train, y01723_train)
    y01723_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr01723.predict(X01723_test)], index = X01723_test.index.values)

    regr106 = model
    regr106.fit(X106_train, y106_train)
    y106_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr106.predict(X106_test)], index = X106_test.index.values)

    regr168 = model
    regr168.fit(X168_train, y168_train)
    y168_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr168.predict(X168_test)], index = X168_test.index.values)

    regr1811 = model
    regr1811.fit(X1811_train, y1811_train)
    y1811_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr1811.predict(X1811_test)], index = X1811_test.index.values)

    regr11116 = model
    regr11116.fit(X11116_train, y11116_train)
    y11116_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11116.predict(X11116_test)], index = X11116_test.index.values)

    regr11618 = model
    regr11618.fit(X11618_train, y11618_train)
    y11618_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11618.predict(X11618_test)], index = X11618_test.index.values)

    regr11823 = model
    regr11823.fit(X11823_train, y11823_train)
    y11823_pred = pd.Series([1 if (i < 0 or i > 1000) else math.ceil(i) for i in regr11823.predict(X11823_test)], index = X11823_test.index.values)
    
    y_pred = pd.DataFrame(columns = list(y007_pred))
    y_pred = pd.concat([y007_pred, y0710_pred, y01017_pred, y01723_pred, y106_pred, y168_pred, y1811_pred, y11116_pred, y11618_pred, y11823_pred]).sort_index()
    
#     for i in y_test.index.values:
#         print((y_test[i], y_pred[i]))
    
    return r2_score(y_test, y_pred), y_pred
    

In [70]:
model = make_pipeline(
    PolynomialFeatures(degree=1, include_bias=False), 
    LinearRegression(n_jobs=-1))

In [71]:
# 8e
score, y_preds = score_multiple4(train, test, model)
score

0.8682736967439065